# INF161 project - Data prep

In [401]:
import pandas as pd
import numpy as np
import plotly.express as px


## Step 1: Read them files

In [402]:
def htmlToDataframe(fileName : str, year : str):
    read = pd.read_html(f"{year}/{fileName}.xls", encoding="UTF-8")
    return pd.DataFrame(read[0])


# Proof of concept, just to have a bit of a gander.
fil = pd.read_html("2018/games.xls", encoding="UTF-8")

fil = pd.DataFrame(fil[0])

[print(x) for x in fil["Notes"].isna() if not x]


[]

### Looks good, now to save the data as CVS files for modelling, rather than HTML files.

In [403]:


years = ["2017", "2018", "2019"]
filenames = ["games", "player-stats", "table", "team-stats"]
games = []
player_stats = []
table = []
team_stats = []

for year in years:
    games.append(htmlToDataframe("games", year))
    player_stats.append(htmlToDataframe("player-stats", year))
    table.append(htmlToDataframe("table", year))
    team_stats.append(htmlToDataframe("team-stats", year))
    #htmlToDataframe(fil, year).to_csv(f"prepared/{year}/{fil}.csv")




## Time to have a bit of a gander at these files.  
First off is games.

In [404]:
games[0]


,Wk,Day,Date,Time,Home,Score,Away,Attendance,Venue,Referee,Match Report,Notes
0,1.0,Mon,2017-04-17,13:00,Medkila,1–1,Sandviken,307.0,Harstad Stadion,Sarah Fatemeh Zangeneh,Match Report,NaN
1,1.0,Mon,2017-04-17,14:00,Avaldsnes,2–1,Vålerenga,346.0,Avaldsnes Idrettssenter,Marit Folstad,Match Report,NaN
2,1.0,Mon,2017-04-17,15:00,Grand Bodø,2–2,Arna-Bjørnar,290.0,Nordlandshallen,Henrikke Holm Nervik,Match Report,NaN
3,1.0,Mon,2017-04-17,15:00,Røa,1–2,Stabæk,348.0,Røabanen,Emilie Dokset,Match Report,NaN
4,1.0,Mon,2017-04-17,15:00,LSK Kvinner,6–1,Klepp,280.0,LSK-hallen,Ingvild Aarland,Match Report,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
155,22.0,Sat,2017-11-04,14:00,Stabæk,5–2,Trondheims-Ørn,217.0,Nadderud Stadion,Marte Sørø,Match Report,NaN
156,22.0,Sat,2017-11-04,14:00,Avaldsnes,5–0,Medkila,241.0,Avaldsnes Idrettssenter,Sarah Fatemeh Zangeneh,Match Report,NaN
157,22.0,Sat,2017-11-04,14:00,Kolbotn,2–0,Grand Bodø,97.0,Sofiemyr stadion,Emilie Dokset,Match Report,NaN
158,22.0,Sat,2017-11-04,14:00,Sandviken,0–2,Røa,102.0,Stemmemyren kunstgressbane,Marit Folstad,Match Report,NaN


For games, we have a lot of features that will either useless, will not affect the outcome of a match or be insignificant enough that they will only cause overfitting on the training data. These include dates, attendance venue, referee, match report and Notes. Let's purge them:

In [405]:
for i in games:
    df = games.pop(0)
    df = df.drop(["Wk", "Day", "Date", "Time", "Attendance", "Venue", "Referee", "Match Report", "Notes"], axis=1)
    if len(df.columns)>3: # Cheeky extra column in 2019
        df = df.drop(["Round"], axis=1)
    games.append(df.dropna(how="all"))
games[0]

,Home,Score,Away
0,Medkila,1–1,Sandviken
1,Avaldsnes,2–1,Vålerenga
2,Grand Bodø,2–2,Arna-Bjørnar
3,Røa,1–2,Stabæk
4,LSK Kvinner,6–1,Klepp
...,...,...,...
155,Stabæk,5–2,Trondheims-Ørn
156,Avaldsnes,5–0,Medkila
157,Kolbotn,2–0,Grand Bodø
158,Sandviken,0–2,Røa


## Next is player stats.

In [406]:
player_stats[0]

Unnamed: 0_level_0          Unnamed: 1_level_0 Unnamed: 2_level_0  \
                    Rk                      Player             Nation   
0                    1      Marianne Aamot Linaker             no NOR   
1                    2           Meryll Abrahamsen             no NOR   
2                    3                Julie Adserø             no NOR   
3                    4         Ine Agnethe Aarskog             no NOR   
4                    5                Teigen Allen             au AUS   
..                 ...                         ...                ...   
255                256            Lisa-Marie Woods             no NOR   
256                257               Kirsty Yallop             nz NZL   
257                258  Gunnhildur Yrsa Jónsdóttir             is ISL   
258                259              Emilie Østerås             no NOR   
259                260                  Vilde Øyri             no NOR   

    Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0  \
                   Pos              Squad                Age   
0                   DF            Medkila               34.0   
1                   DF          Avaldsnes               19.0   
2                   MF     Trondheims-Ørn               25.0   
3                   DF              Klepp               17.0   
4                   DF          Vålerenga               22.0   
..                 ...                ...                ...   
255                 MF          Avaldsnes               32.0   
256                 MF              Klepp               30.0   
257                 MF          Vålerenga               28.0   
258                 MF          Vålerenga               14.0   
259                 FW          Sandviken               22.0   

    Unnamed: 6_level_0 Playing Time               ... Performance             \
                  Born           MP Starts   Min  ...          PK PKatt CrdY   
0               1982.0            4      4   359  ...           0     0    0   
1               1997.0           17     11   945  ...           0     0    4   
2               1991.0            9      9   791  ...           0     0    1   
3               2000.0            3      1   165  ...           0     0    0   
4               1994.0            4      3   266  ...           0     0    1   
..                 ...          ...    ...   ...  ...         ...   ...  ...   
255             1984.0            8      8   645  ...           0     0    0   
256             1986.0           22     22  1926  ...           2     2    2   
257             1988.0           21     21  1890  ...           0     0    1   
258             2002.0            2      0    52  ...           0     0    0   
259             1994.0           10      3   374  ...           0     0    0   

         Per 90 Minutes                      Unnamed: 23_level_0  
    CrdR            Gls Ast G+A  G-PK G+A-PK             Matches  
0      0           0.00 NaN NaN  0.00    NaN             Matches  
1      0           0.10 NaN NaN  0.10    NaN             Matches  
2      0           0.57 NaN NaN  0.57    NaN             Matches  
3      0           0.00 NaN NaN  0.00    NaN             Matches  
4      1           0.00 NaN NaN  0.00    NaN             Matches  
..   ...            ...  ..  ..   ...    ...                 ...  
255    0           0.00 NaN NaN  0.00    NaN             Matches  
256    0           0.14 NaN NaN  0.05    NaN             Matches  
257    0           0.24 NaN NaN  0.24    NaN             Matches  
258    0           1.73 NaN NaN  1.73    NaN             Matches  
259    0           0.24 NaN NaN  0.24    NaN             Matches  

[260 rows x 24 columns]

Here there are a few columns with missing data and some with useless/insignificant data as well. Namely "Nation", "Age", "Pos", "Born" all of "Performance", "Ast", "G+A", "G+A-PK", "G-PK", "Starts", "90s", "Matches".

In [407]:
for i in player_stats:
    df = player_stats.pop(0)
    df.columns = df.columns.droplevel(0)
    player_stats.append(df.drop(["Nation", "Age", "Pos", "PK", "Born", "Starts", "G-PK", "90s", "PKatt", "CrdY", "CrdR", "Ast", "G+A", "G+A-PK", "Matches"], axis=1))
player_stats[0]

,Rk,Player,Squad,MP,Min,Gls,Gls
0,1,Marianne Aamot Linaker,Medkila,4,359,0,0.00
1,2,Meryll Abrahamsen,Avaldsnes,17,945,1,0.10
2,3,Julie Adserø,Trondheims-Ørn,9,791,5,0.57
3,4,Ine Agnethe Aarskog,Klepp,3,165,0,0.00
4,5,Teigen Allen,Vålerenga,4,266,0,0.00
...,...,...,...,...,...,...,...
255,256,Lisa-Marie Woods,Avaldsnes,8,645,0,0.00
256,257,Kirsty Yallop,Klepp,22,1926,3,0.14
257,258,Gunnhildur Yrsa Jónsdóttir,Vålerenga,21,1890,5,0.24
258,259,Emilie Østerås,Vålerenga,2,52,1,1.73


## Next up: Team stats

In [408]:
team_stats[0]

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                Squad               # Pl                Age   
0        Arna-Bjørnar                 22               22.0   
1           Avaldsnes                 23               25.5   
2          Grand Bodø                 22               22.7   
3               Klepp                 19               23.5   
4             Kolbotn                 21               21.9   
5         LSK Kvinner                 18               24.2   
6             Medkila                 26               20.9   
7                 Røa                 22               22.8   
8           Sandviken                 22               24.4   
9              Stabæk                 20               25.5   
10     Trondheims-Ørn                 22               22.1   
11          Vålerenga                 23               25.8   

   Unnamed: 3_level_0 Playing Time                    Performance              \
                 Poss           MP Starts   Min   90s         Gls Ast G-PK PK   
0                 NaN           22    242  1980  22.0          37 NaN   36  1   
1                 NaN           22    242  1980  22.0          49 NaN   48  1   
2                 NaN           22    242  1980  22.0          30 NaN   29  1   
3                 NaN           22    242  1980  22.0          33 NaN   31  2   
4                 NaN           22    242  1980  22.0          16 NaN   16  0   
5                 NaN           22    242  1980  22.0          67 NaN   65  2   
6                 NaN           22    242  1980  22.0           9 NaN    9  0   
7                 NaN           22    242  1980  22.0          42 NaN   42  0   
8                 NaN           22    242  1980  22.0          23 NaN   23  0   
9                 NaN           22    242  1980  22.0          44 NaN   44  0   
10                NaN           22    242  1980  22.0          36 NaN   36  0   
11                NaN           22    242  1980  22.0          38 NaN   38  0   

                   Per 90 Minutes                       
   PKatt CrdY CrdR            Gls Ast G+A  G-PK G+A-PK  
0      1   20    1           1.68 NaN NaN  1.64    NaN  
1      1   22    1           2.23 NaN NaN  2.18    NaN  
2      1   19    0           1.36 NaN NaN  1.32    NaN  
3      2   13    2           1.50 NaN NaN  1.41    NaN  
4      0    8    0           0.73 NaN NaN  0.73    NaN  
5      2   11    0           3.05 NaN NaN  2.95    NaN  
6      0   13    2           0.41 NaN NaN  0.41    NaN  
7      0   22    1           1.91 NaN NaN  1.91    NaN  
8      0   10    0           1.05 NaN NaN  1.05    NaN  
9      0    4    0           2.00 NaN NaN  2.00    NaN  
10     0    9    0           1.64 NaN NaN  1.64    NaN  
11     0   16    2           1.73 NaN NaN  1.73    NaN

This data shares a lot of columns with the player-stats file, and we will remove a lot of the same features.

In [409]:
for i in team_stats:
    df = team_stats.pop(0)
    df.columns = df.columns.droplevel(0)
    team_stats.append(df.drop(["Poss", "Age", "# Pl", "PK", "MP", "Starts", "Min", "90s", "PKatt", "CrdR", "Ast", "G+A", "G+A-PK"], axis=1))
team_stats[0]

,Squad,Gls,G-PK,CrdY,Gls,G-PK
0,Arna-Bjørnar,37,36,20,1.68,1.64
1,Avaldsnes,49,48,22,2.23,2.18
2,Grand Bodø,30,29,19,1.36,1.32
3,Klepp,33,31,13,1.50,1.41
4,Kolbotn,16,16,8,0.73,0.73
5,LSK Kvinner,67,65,11,3.05,2.95
6,Medkila,9,9,13,0.41,0.41
7,Røa,42,42,22,1.91,1.91
8,Sandviken,23,23,10,1.05,1.05
9,Stabæk,44,44,4,2.00,2.00


## Table

In [410]:
table[0]

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Attendance,Top Team Scorer,Goalkeeper,Notes
0,1,LSK Kvinner,22,19,2,1,68,18,50,59,265,Guro Reiten - 18,Cecilie Fiskerstrand,→ Champions League via league finish
1,2,Avaldsnes,22,15,3,4,51,17,34,48,245,"Cecilie Pedersen, Elise Thorsnes - 11",Katie Fraine,→ Champions League via league finish
2,3,Stabæk,22,13,4,5,46,23,23,43,193,Melissa Bjånesøy - 11,Ingrid Hjelmseth,NaN
3,4,Klepp,22,12,4,6,34,24,10,40,267,Tameka Butt - 14,Oda Bogstad,NaN
4,5,Røa,22,11,4,7,42,28,14,37,190,Lisa-Marie Karlseng Utland - 17,Kirvil Schau Odden,NaN
5,6,Arna-Bjørnar,22,9,8,5,39,28,11,35,194,Josée Nahi - 7,Alyssa Giannetti,NaN
6,7,Vålerenga,22,10,4,8,38,33,5,34,449,"Elise Krieghoff, Maren Hauge - 7",Michelle Betos,NaN
7,8,Trondheims-Ørn,22,8,6,8,36,36,0,30,240,Rakel Engesvik - 8,Kristine Nøstmo,NaN
8,9,Sandviken,22,4,6,12,23,34,-11,18,236,Sofie Jensen - 6,Nora Neset Gjøen,NaN
9,10,Kolbotn,22,4,3,15,17,43,-26,15,193,Juliette Kemppi - 5,Aurora Mikalsen,NaN


What a mess! Let's clean this up.

In [411]:
for i in table:
    df = table.pop(0)
    table.append(df.drop(["MP", "GF", "GA", "Attendance", "Notes"], axis=1))
table[0]

,Rk,Squad,W,D,L,GD,Pts,Top Team Scorer,Goalkeeper
0,1,LSK Kvinner,19,2,1,50,59,Guro Reiten - 18,Cecilie Fiskerstrand
1,2,Avaldsnes,15,3,4,34,48,"Cecilie Pedersen, Elise Thorsnes - 11",Katie Fraine
2,3,Stabæk,13,4,5,23,43,Melissa Bjånesøy - 11,Ingrid Hjelmseth
3,4,Klepp,12,4,6,10,40,Tameka Butt - 14,Oda Bogstad
4,5,Røa,11,4,7,14,37,Lisa-Marie Karlseng Utland - 17,Kirvil Schau Odden
5,6,Arna-Bjørnar,9,8,5,11,35,Josée Nahi - 7,Alyssa Giannetti
6,7,Vålerenga,10,4,8,5,34,"Elise Krieghoff, Maren Hauge - 7",Michelle Betos
7,8,Trondheims-Ørn,8,6,8,0,30,Rakel Engesvik - 8,Kristine Nøstmo
8,9,Sandviken,4,6,12,-11,18,Sofie Jensen - 6,Nora Neset Gjøen
9,10,Kolbotn,4,3,15,-26,15,Juliette Kemppi - 5,Aurora Mikalsen


Marvelous. For simplicity, lets add all the years together as well and save them all to new, prepared data files.

In [412]:
for yearIndex in range(len(years)):
    games[yearIndex].to_csv(f"prepared/{years[yearIndex]}/games.csv")
    player_stats[yearIndex].to_csv(f"prepared/{years[yearIndex]}/player-stats.csv")
    table[yearIndex].to_csv(f"prepared/{years[yearIndex]}/table.csv")
    team_stats[yearIndex].to_csv(f"prepared/{years[yearIndex]}/team-stats.csv")

In [413]:
# Concat, where applicable.

games_All = pd.concat(games)
player_stats_All = pd.concat(player_stats)
table_All = pd.concat(table)
team_stats_All = pd.concat(team_stats)

games_All.to_csv(f"prepared/combined/games-all.csv")
player_stats_All.to_csv(f"prepared/combined/player-stats-all.csv")
table_All.to_csv(f"prepared/combined/table-all.csv")
team_stats_All.to_csv(f"prepared/combined/team-stats-all.csv")
